In [5]:
import dgl.nn as gnn
import dgl.function as fn

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from utils import create_edge_pred_graph, create_heterograph

# Homogenious graph edge regression

Стандартная история - предсказать что-нибудь по поводу узлов на основе их скрытого представления, полученного после нескольких слоев GNN. Предсказания на уровне связи можно строить на основе представлений инцидентных этой связи узлов (и, быть может, фичей самой связи).

In [19]:
n_nodes, n_edges, n_node_features,n_edge_features = 50, 100, 10, 10
G = create_edge_pred_graph(n_nodes, n_edges, n_node_features, n_edge_features)

Вариант 1. Нужно получить одно число для каждого ребра

In [4]:
class DotProductPredictor(nn.Module):
    def forward(self, G, features):
        with G.local_scope():
            G.ndata['h'] = features
            G.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            return G.edata['score']
            

In [8]:
predictor = DotProductPredictor()
scores = predictor(G, G.ndata['feature'])
scores.shape

torch.Size([100, 1])

Вариант 2. Нужно получить вектор для каждого ребра

In [11]:
class MLPPredictor(nn.Module):
    def __init__(self, n_node_features, n_edge_features):
        super().__init__()
        self.linear = nn.Linear(2 * n_node_features, n_edge_features)

    def gen_edge_feature(self, edges):
        src = edges.src['h']
        dst = edges.dst['h']
        src_dst = torch.cat([src, dst], dim=1)
        edge_feature = self.linear(src_dst)
        return {'e_h': edge_feature}

    def forward(self, G, features):
        with G.local_scope():
            G.ndata['h'] = features
            G.apply_edges(self.gen_edge_feature)
            return G.edata['e_h']
            

In [20]:
predictor = MLPPredictor(n_node_features, 17)
features = predictor(G, G.ndata['feature'])
features.shape

torch.Size([100, 17])

Полный цикл обучения для предсказания 1 числа на каждом ребре

In [23]:
class GCN(nn.Module):
    def __init__(self, n_inputs, n_hidden):
        super().__init__()
        self.conv1 = gnn.SAGEConv(n_inputs, n_hidden, aggregator_type='mean', activation=F.relu)
        self.predictor = DotProductPredictor()

    def forward(self, G, features):
        out = self.conv1(G, features)
        out = self.predictor(G, out)
        return out

In [35]:
node_features = G.ndata['feature']
edge_labels = G.edata['label']
train_mask = G.edata['train_mask']


model = GCN(n_inputs=n_node_features, n_hidden=32)
optimizer = optim.Adam(model.parameters(), lr=.01)
criterion = nn.MSELoss()

for epoch in range(21):
    # forward
    preds = model(G, node_features).flatten()
    loss = criterion(preds[train_mask], edge_labels[train_mask])
    # backward
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if not epoch % 5:
        print(f'Epoch #{epoch} loss={loss.item()}')

Epoch #0 loss=69.82396697998047
Epoch #5 loss=17.834178924560547
Epoch #10 loss=5.584915637969971
Epoch #15 loss=2.6153640747070312
Epoch #20 loss=1.710088849067688


# Heterogenious graph

Для гетерографов процесс похожий, только нужно сгенерировать представления всех узлов _всех типов_, а затем получить представления для _нужного типа_ ребер при помощи `apply_edges`.

In [3]:
class HeteroDotProductPredictor(nn.Module):
    def forward(self, G, features, etype):
        # features - это представления узлов для всех типов
        with G.local_scope():
            # таким образом можно присвоить свойство h всем типам узлов сразу
            G.ndata['h'] = features
            G.apply_edges(fn.u_dot_v('h', 'h', 'score'), etype=etype)
            return G.edges[etype].data['score']

In [6]:
G = create_heterograph()

In [12]:
predictor = HeteroDotProductPredictor()
features = {ntype: G.nodes[ntype].data['feature'] for ntype in G.ntypes}
out = predictor(G, features, etype='click')
out.shape

torch.Size([5000, 1])

In [18]:
class GCN(nn.Module):
    def __init__(self, n_inputs, n_hidden, rel_names):
        super().__init__()
        conv1_modules = {rel: gnn.GraphConv(n_inputs, n_hidden) for rel in rel_names}
        self.conv1 = gnn.HeteroGraphConv(conv1_modules, aggregate='sum')
        self.predictor = HeteroDotProductPredictor()

    def forward(self, G, features, etype):
        # HeteroGraphConv принимает на вход словарь тип отношения: фичи узлов и 
        # возвращает словарь такой же структуры
        out = self.conv1(G, features)
        out = self.predictor(G, out, etype)
        return out

In [23]:
n_hetero_features = 10
rel_names = G.etypes
features = {ntype: G.nodes[ntype].data['feature'] for ntype in G.ntypes}
labels = G.edges['click'].data['label']
train_mask = G.edges['click'].data['train_mask']

model = GCN(n_inputs=n_hetero_features, n_hidden=32, rel_names=rel_names)
optimizer = optim.Adam(model.parameters(), lr=.01)
criterion = nn.MSELoss()

for epoch in range(21):
    # forward
    preds = model(G, features, 'click').flatten()
    loss = criterion(preds[train_mask], labels[train_mask])
    # backward
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if not epoch % 5:
        print(f'Epoch #{epoch} loss={loss.item()}')


Epoch #0 loss=36.21980667114258
Epoch #5 loss=26.871191024780273
Epoch #10 loss=15.392216682434082
Epoch #15 loss=8.76208782196045
Epoch #20 loss=8.20315170288086
